In [1]:
%pip install pathlib
%pip install jinja2
%pip install git+git://github.com/PythonicNinja/pydrill.git

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Cloning git://github.com/PythonicNinja/pydrill.git to /tmp/pip-req-build-tvf8mf_r
  Running command git clone -q git://github.com/PythonicNinja/pydrill.git /tmp/pip-req-build-tvf8mf_r
Note: you may need to restart the kernel to use updated packages.


In [1]:
!pip install presto-python-client

In [2]:
from pathlib import Path
import jinja2
import prestodb.dbapi as presto
from pydrill.client import PyDrill
import pandas as pd
import time
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [3]:
# configure sources
sources={
    "li_source":"pg1",
    "o_source":"pg2",
    "c_source":"pg3",
    "pa_source":"pg4",
    "s_source":"pg1",
    "ps_source":"pg2",
    "n_source":"pg3",
    "r_source":"pg4",
}

def get_sources_config(scale,**kwargs):
    """Returns a dict with tabels maped to sources"""
    mapped = {
        "lineitem":f"{kwargs['li_source']}.public.pg_{scale}_lineitem",
        "orders":f"{kwargs['o_source']}.public.pg_{scale}_orders",
        "customer":f"{kwargs['c_source']}.public.pg_{scale}_customer",
        "part":f"{kwargs['pa_source']}.public.pg_{scale}_part",
        "supplier":f"{kwargs['s_source']}.public.pg_{scale}_supplier",
        "partsupp":f"{kwargs['ps_source']}.public.pg_{scale}_partsupp",
        "nation":f"{kwargs['n_source']}.public.pg_{scale}_nation",
        "region":f"{kwargs['r_source']}.public.pg_{scale}_region"
    }
    return mapped


config = get_sources_config(scale="sf1",**sources)

In [8]:
import os
os.getcwd()

'/home/jovyan'

In [9]:
!ls

benchmark.ipynb  work


In [17]:
def get_queries():
    return [q for q in Path("./queries").iterdir()]

def render_queries(sources_config,queries_path):
    """Returns a dict where the key is the tpch query name and the value is the rendered query"""
    result = {}
    for path in queries_path:
        key = path.name.split(".")[0]
        value = jinja2.Template(path.read_text()).render(**sources_config)
        result[key]=value
    return result

query_dict = render_queries(sources_config=config,queries_path=get_queries())

In [18]:
class Presto():
    def __init__(self,host="presto",port=8080,user="demo"):
        self.conn = presto.Connection(host="presto", port=8080, user="demo")
        self.cur = self.conn.cursor()
    
    def run_query(self,sql,show=False):
        self.cur.execute(sql)
        result = self.cur.fetchall()
        if show:
            return result

In [19]:
class Drill(PyDrill):
    def __init__(self,**kwargs):
        PyDrill.__init__(self, **kwargs)
        
    def run_query(self,sql,timeout=100000,show=False):
        if show:
            return self.query(sql,timeout=timeout).to_dataframe()
        return self.query(sql,timeout=timeout)

In [20]:
conn = presto.Connection(host="presto", port=8080, user="demo")
cur = conn.cursor()

In [23]:
cur.execute('''
 SELECT
     l_orderkey,
     sum(l_extendedprice * (1 - l_discount)) as revenue,
     o_orderdate,
     o_shippriority
 FROM
     pg2.public.pg_sf1_customer,
     pg3.public.pg_sf1_orders,
     pg3.public.pg_sf1_lineitem
 WHERE
     c_mktsegment = 'BUILDING'
     AND c_custkey = o_custkey
     AND l_orderkey = o_orderkey
     AND o_orderdate < date '1995-03-15'
     AND l_shipdate > date '1995-03-15'
 GROUP BY
     l_orderkey,
     o_orderdate,
     o_shippriority
 ORDER BY
     revenue desc,
     o_orderdate
 LIMIT 10
 ''')

In [24]:
presto_res = cur.fetchall()

In [25]:
presto_df = pd.DataFrame(presto_res,columns=["l_orderkey","revenue","o_orderdate","o_shippriority"])

In [26]:
presto_df.head()

,l_orderkey,revenue,o_orderdate,o_shippriority
0,2456423,3249448.0888,1995-03-05,0
1,3459808,3246709.5912,1995-03-04,0
2,492164,3122592.4880,1995-02-19,0
3,1188320,3076303.4872,1995-03-09,0
4,2435712,3029384.4464,1995-02-26,0
